In [50]:
import pandas as pd
import sys

sys.path.append("..")

from functions import (df_security_code, 
        print_shape
)

In [53]:
data = pd.read_csv('../data/a_price_financial_fill.csv', parse_dates=['Date'],  index_col=[0])
df = df_security_code(data)
del data

In [55]:
print_shape(df, False)

 Shape:
 ----------------------------------------
 Observations:   1.2K
 Features:       53
 Feature Date:    datetime64[ns]
----------------------------------------
Index(['RowId', 'Date', 'SecuritiesCode', 'AdjustmentFactor',
       'ExpectedDividend', 'SupervisionFlag', 'Target', 'ad_Open', 'ad_High',
       'ad_Low', 'ad_Close', 'ad_Volume', 'ad_Target', 'ad_Close_lag1',
       'ad_Close_sma10', 'ad_Open_lag1', 'ad_Open_sma10', 'ad_High_lag1',
       'ad_High_sma10', 'ad_Low_lag1', 'ad_Low_sma10', 'ad_Volume_lag1',
       'ad_Volume_sma10', 'RSI', 'Return', 'logreturn', 'Log_Return', 'MACD',
       'MACD_h', 'MACD_s', 'Year', 'week', 'Volatility_week', 'macd', 'macd_h',
       'macd_s', 'Date_f_', 'Day', 'Month', 'Year_f_', 'SecuritiesCode_f_',
       'Profit', 'NetSales', 'margin', 'profit_ttm', 'rev_ttm',
       'win_quarter_growth', 'rev_quarter_growth', 'win_yoy_growth',
       'rev_yoy_growth', 'win_ttm_growth', 'rev_ttm_growth', 'margin_growth'],
      dtype='object')
-------

In [57]:
feat_not_to_scale = ['RowId', 'Date', 'SecuritiesCode', 'Target','Log_Return', 'MACD',
       'MACD_h', 'MACD_s', 'Year', 'week','Date_f_', 'Day', 'Month', 'Year_f_', 'SecuritiesCode_f_' ]

feat_to_scale = [x for x in df.columns if x not in feat_not_to_scale]
len(feat_to_scale)

38